In [1]:
!pip install --upgrade tables

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4


In [2]:
!pip install eli5

     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

## Wczytujemy dane

In [16]:
df = pd.read_hdf("data/car.h5")
df.shape

(106494, 155)

In [22]:
df.columns.values

array(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'price_value', 'seller_address', 'seller_name', 'seller_type',
       'feature_czujniki-parkowania-przednie',
       'feature_poduszka-powietrzna-chroniąca-kolana',
       'feature_kurtyny-powietrzne', 'feature_klimatyzacja-dwustrefowa',
       'feature_światła-led', 'feature_czujnik-zmierzchu',
       'feature_elektrycznie-ustawiane-lusterka',
       'feature_asr-(kontrola-trakcji)',
       'feature_poduszka-powietrzna-kierowcy', 'feature_cd',
       'feature_elektryczne-szyby-przednie',
       'feature_poduszka-powietrzna-pasażera',
       'feature_system-start-stop', 'feature_światła-do-jazdy-dziennej',
       'feature_komputer-pokładowy', 'feature_elektryczne-szyby-tylne',
       'feature_klimatyzacja-manualna', 'feature_tapicerka-welurowa',
       'feature_czujnik-deszczu', 'feature_światła-przeciwmgielne',
       'feature_ogrzewanie-postojowe', 'feature_radio-niefabryczne',
       'feature_regulowane-zaw

## Dummy Model

In [23]:
df.select_dtypes(np.number).columns

Index(['price_value', 'car_id'], dtype='object')

In [25]:
X = df[['car_id']].values
y = df['price_value'].values

model = DummyRegressor()
model.fit(X ,y)

DummyRegressor(constant=None, quantile=None, strategy='mean')

In [26]:
y_pred = model.predict(X)
mae(y, y_pred=y_pred)

39465.934630440985

In [27]:
[x for x in df.columns if 'price' in x]

['price_currency', 'price_details', 'price_value']

In [34]:
df.price_currency.value_counts(normalize=True) *100

PLN    100.0
Name: price_currency, dtype: float64

In [0]:
df = df [df.price_currency != 'EUR']

In [33]:
df.shape

(106290, 155)

## Cechy



In [0]:
SUFFIX_CAT = '__cat'

In [48]:
for feat in df.columns:
  if  isinstance(df[feat][0], list) == False and SUFFIX_CAT not in feat:
    df[feat + SUFFIX_CAT] = df[feat].factorize()[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [49]:
df.columns.values.size

309

In [0]:
feats_cat = [x for x in df.columns if SUFFIX_CAT in x]
feats_cat = [x for x in feats_cat if 'price' not in x]

In [55]:
len(feats_cat)

151

In [0]:
X = df[feats_cat].values
y = df['price_value'].values

model = DecisionTreeRegressor(max_depth=8)
scores = cross_val_score(model,X, y, cv=3, scoring='neg_mean_absolute_error')

In [70]:
np.mean(scores)

-16612.208135554363

In [72]:
m = DecisionTreeRegressor(max_depth=8)
m.fit(X,y)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=8,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [74]:
imp = PermutationImportance(m, random_state=254).fit(X,y)
eli5.show_weights(imp, feature_names = feats_cat)


Weight,Feature
0.3101 ± 0.0134,param_rok-produkcji__cat
0.2934 ± 0.0062,param_napęd__cat
0.2715 ± 0.0089,param_stan__cat
0.2291 ± 0.0095,param_faktura-vat__cat
0.0988 ± 0.0026,param_moc__cat
0.0587 ± 0.0021,param_skrzynia-biegów__cat
0.0527 ± 0.0057,param_marka-pojazdu__cat
0.0457 ± 0.0013,param_pojemność-skokowa__cat
0.0358 ± 0.0029,feature_kamera-cofania__cat
0.0246 ± 0.0027,param_wersja__cat


In [77]:
new_features = ['param_rok-produkcji__cat', 'param_napęd__cat', 'param_stan__cat', 
                'param_faktura-vat__cat','param_moc__cat', 'param_skrzynia-biegów__cat',
                'param_marka-pojazdu__cat', 'param_pojemność-skokowa__cat', 'feature_kamera-cofania__cat',
                'param_wersja__cat', 'feature_łopatki-zmiany-biegów__cat', 'param_model-pojazdu__cat',
                'param_kod-silnika__cat', 'feature_światła-led__cat', ]
X = df[new_features].values
y = df['price_value'].values

model = DecisionTreeRegressor(max_depth=8)
scores = cross_val_score(model,X, y, cv=3, scoring='neg_mean_absolute_error')
np.mean(scores)

-16898.257558273708